In [5]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [6]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import cifar10

In [ ]:
# Reference
# http://karpathy.github.io/2011/04/27/manually-classifying-cifar10/

In [7]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# convert the data types to float 32 bit & then normalize between 0~1
x_train = x_train.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0

170508288/170498071 [==============================] - 2s 0us/step


# Implement with Sequential API

In [ ]:
model = keras.Sequential(
    [
     keras.Input(shape=(32, 32, 3)),
     layers.Conv2D(32, 3, padding='valid', activation='relu'),
     layers.MaxPool2D(pool_size=(2,2)),
     layers.Conv2D(64, 3, activation='relu'),
     layers.MaxPool2D(),
     layers.Conv2D(128, 3, activation='relu'),
     layers.Flatten(),
     layers.Dense(64, activation='relu'),
     layers.Dense(10)
    ]
)

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 30, 30, 32)        896       
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 15, 15, 32)       0         
 2D)                                                             
                                                                 
 conv2d_4 (Conv2D)           (None, 13, 13, 64)        18496     
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 6, 6, 64)         0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 4, 4, 128)         73856     
                                                                 
 flatten_1 (Flatten)         (None, 2048)             

In [ ]:
model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.Adam(lr=3e-4),
    metrics=["accuracy"],
)

model.fit(x_train, y_train, batch_size=64, epochs=10, verbose=2)
model.evaluate(x_test, y_test, batch_size=64, verbose=2)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Epoch 1/10
782/782 - 65s - loss: 1.6626 - accuracy: 0.3941 - 65s/epoch - 83ms/step
Epoch 2/10
782/782 - 70s - loss: 1.3485 - accuracy: 0.5158 - 70s/epoch - 90ms/step
Epoch 3/10
782/782 - 63s - loss: 1.2246 - accuracy: 0.5684 - 63s/epoch - 81ms/step
Epoch 4/10
782/782 - 63s - loss: 1.1226 - accuracy: 0.6064 - 63s/epoch - 81ms/step
Epoch 5/10
782/782 - 63s - loss: 1.0377 - accuracy: 0.6361 - 63s/epoch - 80ms/step
Epoch 6/10
782/782 - 64s - loss: 0.9755 - accuracy: 0.6598 - 64s/epoch - 81ms/step
Epoch 7/10
782/782 - 63s - loss: 0.9234 - accuracy: 0.6791 - 63s/epoch - 81ms/step
Epoch 8/10
782/782 - 63s - loss: 0.8757 - accuracy: 0.6968 - 63s/epoch - 80ms/step
Epoch 9/10
782/782 - 63s - loss: 0.8305 - accuracy: 0.7124 - 63s/epoch - 81ms/step
Epoch 10/10
782/782 - 63s - loss: 0.7919 - accuracy: 0.7274 - 63s/epoch - 81ms/step
157/157 - 4s - loss: 0.9110 - accuracy: 0.6929 - 4s/epoch - 23ms/step


[0.9110416769981384, 0.6929000020027161]

In [ ]:
# the above model has just 3 convolution layers plus 10 epochs & with this, we see around 73% of training accuracy & around 70% accuracy on validation.
# So this accuracy can be increased:
# - increase the number of epochs
# - increase the number of convolution layers

# Implement with Functional API

In [ ]:
# lets do it with function
def functional_model():
  inputs = keras.Input(shape=(32, 32, 3))
  x = layers.Conv2D(32, 3)(inputs)
  x = layers.BatchNormalization()(x)
  x = keras.activations.relu(x)
  x = layers.MaxPool2D()(x)
  x = layers.Conv2D(64, 5, padding='same')(x)
  x = layers.BatchNormalization()(x)
  x = keras.activations.relu(x)
  x = layers.Conv2D(128, 3)(x)
  x = layers.BatchNormalization()(x)
  x = keras.activations.relu(x)
  x = layers.Flatten()(x)
  x = layers.Dense(64, activation='relu')(x)
  outputs = layers.Dense(10)(x)
  model = keras.Model(inputs=inputs, outputs=outputs)
  return model


In [ ]:
model = functional_model()

model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.Adam(lr=3e-4),
    metrics=["accuracy"],
)

model.fit(x_train, y_train, batch_size=64, epochs=10, verbose=2)
model.evaluate(x_test, y_test, batch_size=64, verbose=2)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Epoch 1/10
782/782 - 254s - loss: 1.3255 - accuracy: 0.5254 - 254s/epoch - 325ms/step
Epoch 2/10
782/782 - 254s - loss: 0.9069 - accuracy: 0.6816 - 254s/epoch - 325ms/step
Epoch 3/10
782/782 - 258s - loss: 0.7493 - accuracy: 0.7366 - 258s/epoch - 330ms/step
Epoch 4/10
782/782 - 251s - loss: 0.6430 - accuracy: 0.7749 - 251s/epoch - 321ms/step
Epoch 5/10
782/782 - 252s - loss: 0.5542 - accuracy: 0.8081 - 252s/epoch - 322ms/step
Epoch 6/10
782/782 - 252s - loss: 0.4740 - accuracy: 0.8360 - 252s/epoch - 322ms/step
Epoch 7/10
782/782 - 254s - loss: 0.4096 - accuracy: 0.8586 - 254s/epoch - 325ms/step
Epoch 8/10
782/782 - 259s - loss: 0.3472 - accuracy: 0.8814 - 259s/epoch - 331ms/step
Epoch 9/10
782/782 - 261s - loss: 0.2877 - accuracy: 0.9052 - 261s/epoch - 334ms/step
Epoch 10/10
782/782 - 256s - loss: 0.2385 - accuracy: 0.9201 - 256s/epoch - 327ms/step
157/157 - 12s - loss: 0.8896 - accuracy: 0.7345 - 12s/epoch - 79ms/step


[0.8895913362503052, 0.734499990940094]

In [ ]:
# with the above functional model, we have introduced the batch normalization and with this we see training accuracy is increased to around 92% but the validation accuracy is around 72%. 
# Here we still see there is gap between training & validation accuracies & see we have overfit in the training & needs to be removed may be by introducind regularization.
# So lets reuse the above functional_model method to introduce the couple of regularization techniques in our model & create a new method for that below.
# we will introduce L2 regularization alogn with dropout in this case.

In [8]:
from tensorflow.keras import regularizers

In [15]:
# introducing L2 & Dropout regularizations

def functional_model_regularization():
  inputs = keras.Input(shape=(32, 32, 3))
  x = layers.Conv2D(32, 3, padding='same', kernel_regularizer=regularizers.l2(0.01))(inputs)
  x = layers.BatchNormalization()(x)
  x = keras.activations.relu(x)
  x = layers.MaxPool2D()(x)
  x = layers.Conv2D(64, 5, padding='same', kernel_regularizer=regularizers.l2(0.01))(x)
  x = layers.BatchNormalization()(x)
  x = keras.activations.relu(x)
  x = layers.Conv2D(128, 3, padding='same', kernel_regularizer=regularizers.l2(0.01))(x)
  x = layers.BatchNormalization()(x)
  x = keras.activations.relu(x)
  x = layers.Flatten()(x)
  x = layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.01))(x)
  x = layers.Dropout(0.5)(x)
  outputs = layers.Dense(10)(x)
  model = keras.Model(inputs=inputs, outputs=outputs)
  return model

In [ ]:
model = functional_model_regularization()

model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.Adam(lr=3e-4),
    metrics=["accuracy"],
)

model.fit(x_train, y_train, batch_size=64, epochs=10, verbose=2)
model.evaluate(x_test, y_test, batch_size=64, verbose=2)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Epoch 1/10
782/782 - 337s - loss: 3.1139 - accuracy: 0.1094 - 337s/epoch - 431ms/step
Epoch 2/10
